In [121]:
import numpy as np
import pandas as pd
import time

from scipy.stats import norm
import scipy.stats as stats

from sklearn.datasets import make_classification

## Generate Data

In [118]:
X, y = make_classification(random_state=42, n_samples=10000, n_features=10, n_redundant=0) # generate a random dataset
y = y.reshape(-1,1) # make Y a column vector

# add a column of ones to X
ones = np.ones((X.shape[0],1))
X = np.hstack((ones,X))

In [120]:
epsilon = 1e-6

# Fischer Scoring

In [126]:
def probit(X, y, epsilon, verbose = True):
    n, p = np.shape(X)
    
    b_0 = np.zeros((p,1)) # Setting initial value of beta as 0
    Xb = np.dot(X,b_0) # Calculating Xb
    mu = norm.cdf(Xb) # Calculating mu
    W = np.diag(((norm.pdf(Xb)**2)/ (mu*(1-mu))).flatten()) # Calculating W
    Z = Xb + (y - mu)/norm.pdf(Xb) # Calculating Z

    iteration = 0 # Set the number of iterations

    start_time = time.process_time() # Start the timer
    while True: ### Algorithm Starts
        iteration += 1

        # Iterative updates
        b = np.linalg.inv(X.T @ W @ X) @ X.T @ W @ Z
        Xb = np.dot(X,b)
        mu = norm.cdf(Xb)
        W = np.diag(((norm.pdf(Xb)**2)/(mu*(1-mu))).flatten())
        Z = Xb + (y - mu)/norm.pdf(Xb)

        # Calculating the convergence criteria
        delta = np.linalg.norm(b-b_0)

        if verbose:
            print(f"Iteration:, {iteration}, Converging = {delta}")

        if delta < epsilon: # Checking for convergence
            print("\nConvergence reached with value:", delta)
            print("Number of iteration:", iteration)
            print(f"Time taken: {time.process_time() - start_time : .2f} seconds")
            break

        b_0 = b 
    
    print(f"\nParameters: {b.flatten()}")
    return(b.flatten())

b = probit(X, y, epsilon, verbose=False)



Convergence reached with value: 1.0491831950805439e-07
Number of iteration: 9
Time taken:  48.62 seconds

Parameters: [-6.66455910e-02  2.31702932e-03 -1.97892969e-02  4.08063752e-01
  3.69850871e-02 -2.35247404e-02 -1.74722354e-02  1.47049619e+00
 -6.92819888e-03 -7.36593907e-04  4.58366051e-03]


# Metropolis Hastings

In [ ]:
EPSILON = 1e-10

def probit_link_function(X, beta):
    linear_pred = np.dot(X, beta)
    return stats.norm.cdf(linear_pred)
def log_likelihood(y, X, beta):
    p = probit_link_function(X, beta)
    p = np.clip(p, EPSILON, 1 - EPSILON)
    log_lik = np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))
    return log_lik
def log_prior(beta):
    prior = stats.norm.logpdf(beta, loc=0, scale=1)
    return np.sum(prior)

def metropolis_hastings(y, X, num_iterations, initial_beta, proposal_sd):
    beta = initial_beta
    accepted_samples = [beta]
    
    for i in range(num_iterations):
        current_log_lik = log_likelihood(y, X, beta)
        current_log_prior = log_prior(beta)
        current_log_posterior = current_log_lik + current_log_prior

        proposed_beta = beta + np.random.normal(0, proposal_sd, size=len(beta))
        
        proposed_log_lik = log_likelihood(y, X, proposed_beta)
        proposed_log_prior = log_prior(proposed_beta)
        proposed_log_posterior = proposed_log_lik + proposed_log_prior
        
        acceptance_ratio = min(1, np.exp(proposed_log_posterior - current_log_posterior))
        
        if acceptance_ratio > np.random.rand():
            beta = proposed_beta
            accepted_samples.append(beta)
        else:
            accepted_samples.append(beta)
    return np.array(accepted_samples)